## App store review scraping

- 구글플레이스토어와 달리, 웹에서 크롤링이 불가 (RSS를 이용하여 웹크롤링 하는  방법이 있으나, 촤대 500개의 리뷰만 가지고 올 수 있음 (참고: [[리뷰 크롤링] AppStore 어플 리뷰 가져오기 3 in python(feat.RSS)](https://signing.tistory.com/51))
- 웹으로 앱스토어에 들어가보면 리뷰는 10개 밖에 안보여줌 (애플에서 일부러 막아둔듯)
- 그렇다면 **웹**이 아닌 **앱**에서 리뷰를 수집 해야하는데 이는 사실상 불가능에 가까움
- RSS 말고 웹에서 리뷰를 수집할 수 있는 또다른 방법은 **app_store_scraper**임
  - 근데 이건 뭔가 API를 call해서 불러와서 그런건지 코드 돌릴 때마다 불러오는 리뷰 개수가 다르고, RSS(최대 500개)보다는 많이 가져오지만 1000~2000개 수준에서 밖에 못가져옴

In [1]:
!pip install app_store_scraper

import pandas as pd
import numpy as np
import json
import time
from tqdm import tqdm
from app_store_scraper import AppStore

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 KB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 KB 7.1 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling chardet-4.0.0:
      Successfully uninstalled chardet-4.0.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.15
    Uninstalling urllib3-1.26.15:
      Successfully uninstalled urllib3-1.26.15
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
      Successfully uninstalled idna-3.4
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      S

In [2]:
def Extract_reviews(app_name:str, app_id:int): 
  ott = AppStore(country='kr', app_name=app_name, app_id=app_id)
  ott.review()

  ott_reviews = pd.DataFrame(ott.reviews).sort_values('date', ascending=False).reset_index(drop=True)
  ott_reviews['ott'] = app_name
  return ott_reviews

In [3]:
ott_platform = {'tiving': 400101401, 'netflix': 363590051}

app_store_reviews = pd.DataFrame()
for i in tqdm(ott_platform):
  ott_reviews = Extract_reviews(i, ott_platform[i])
  app_store_reviews= pd.concat([app_store_reviews, ott_reviews], axis=0)

print('Done..')

100%|██████████| 2/2 [01:56<00:00, 58.34s/it]

Done..


In [6]:
app_store_reviews[app_store_reviews['ott'] == 'tiving']

,date,developerResponse,review,rating,isEdited,title,userName,ott
0,2023-03-29 06:57:41,NaN,안녕하세요 저는 태국에 살고있는 6학년 학생인데 제가 어렸을때부터 진짜 엄청 좋아하...,4,False,도와주세요 ㅜㅜ,아됴우,tiving
1,2023-03-23 18:40:22,"{'id': 35578034, 'body': '나만 안되니 고객님, 안녕하세요. T...",끌 수 있게 해주세요. 손만 잘못 닿으면 앞으로 뒤로 감겨서 짜증나 죽겠어요.\n그...,3,False,제스처모드 좀,나만 안되니,tiving
2,2023-03-23 11:46:38,"{'id': 35601512, 'body': '제임스봉구 고객님, 안녕하세요. TV...",이제 애플티비 tvos 버전이 16.4로 정식 판업 되었으니\nOSD 버그 얼렁 고...,4,False,TVOS 16.4 버그 수정좀,제임스봉구,tiving
3,2023-03-22 11:56:29,"{'id': 35557757, 'body': '볼수가 없슈 고객님, 안녕하세요. T...",한달전쯤부터 어플이 느려서 썸네일 이미지도 안떠요. 카테고리 넘어가는데도 시간 걸리...,3,False,너무 느려서 볼수가 없어요,볼수가 없슈,tiving
4,2023-03-22 04:02:01,"{'id': 35557750, 'body': '복숭아크리스탈 고객님, 안녕하세요. ...",태블릿 어플로는 플레이가 안되고 휴대폰 어플에서는 되길래 큰화면으로 보려고 웹으로 ...,1,False,어플을 잘만들던가 아니면 웹 시청을 막지를 말던가,복숭아크리스탈,tiving
...,...,...,...,...,...,...,...,...
3955,2017-10-06 09:04:53,NaN,캐쉬 이용해서 영화 소장했는데 다운이 너무나 느립니다.\n그냥 재생할려니 화면이 안...,1,False,너무너무느림.,메쥬는쨩,tiving
3956,2017-10-05 16:49:06,NaN,ㅠㅠㅠ와이파이가 문젠가하기엔 다른 영상 보는 어플들은 전혀 문제가 없구요.. 티빙만...,2,False,진짜 뚝뚝 끊기는거 너무 심해요,Jeocn,tiving
3957,2017-09-25 10:59:29,NaN,아이폰 8년 넘게 쓰면서 리뷰 처음 작성하는데 이런 어플은 처음이네요.. 왜 자꾸 ...,1,False,최악,hyojoo kim,tiving
3958,2017-09-20 04:47:48,NaN,재밌는 게 많네요,5,False,좋아요,흑흑ㅁㅁ,tiving
